#Processamento de Linguagem Natuaral
##Previsão do tipo da reclamação a partir do texto escrito pelo consumidor

##Disciplina - Aprendizagem de Máquina - Prof. Lucas Gonçalves
##Aluno - Francisco Flávio Cardoso Gomes

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
data=pd.read_csv('/content/drive/MyDrive/complaints.csv')
data.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc. \nis trying to collect...,NaN,TRANSWORLD SYSTEMS INC,FL,335XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3384392
1,2019-09-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,15206,NaN,Consent not provided,Web,2019-09-20,Closed with non-monetary relief,Yes,NaN,3379500
2,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,"Over the past 2 weeks, I have been receiving e...",NaN,"Diversified Consultants, Inc.",NC,275XX,NaN,Consent provided,Web,2019-11-08,Closed with explanation,Yes,NaN,3433198
3,2019-09-15,Debt collection,Other debt,Attempts to collect debt not owed,Debt was result of identity theft,Pioneer has committed several federal violatio...,NaN,Pioneer Capital Solutions Inc,CA,925XX,NaN,Consent provided,Web,2019-09-15,Closed with explanation,Yes,NaN,3374555
4,2021-01-24,Credit card or prepaid card,General-purpose credit card or charge card,Problem when making payments,Problem during payment process,NaN,NaN,ENCORE CAPITAL GROUP INC.,OR,97501,NaN,NaN,Web,2021-01-24,Closed with explanation,Yes,NaN,4090602


In [11]:
data.isnull().sum()

Date received                         0
Product                               0
Sub-product                      235165
Issue                                 0
Sub-issue                        598707
Consumer complaint narrative    1294836
Company public response         1138807
Company                               0
State                             36141
ZIP code                         188156
Tags                            1709156
Consumer consent provided?       682466
Submitted via                         0
Date sent to company                  0
Company response to consumer          2
Timely response?                      0
Consumer disputed?              1191207
Complaint ID                          0
dtype: int64

In [12]:
data = data.replace(np.nan, '', regex=True)

In [13]:
data.isnull().sum()

Date received                   0
Product                         0
Sub-product                     0
Issue                           0
Sub-issue                       0
Consumer complaint narrative    0
Company public response         0
Company                         0
State                           0
ZIP code                        0
Tags                            0
Consumer consent provided?      0
Submitted via                   0
Date sent to company            0
Company response to consumer    0
Timely response?                0
Consumer disputed?              0
Complaint ID                    0
dtype: int64

In [14]:
data.shape

(1959680, 18)

In [15]:
data["concat"] = data["Issue"]+" " + data["Sub-issue"] +" " +data["Consumer complaint narrative"]


data['concat'].head()

0    Attempts to collect debt not owed Debt is not ...
1    Incorrect information on your report Informati...
2    Communication tactics Frequent or repeated cal...
3    Attempts to collect debt not owed Debt was res...
4    Problem when making payments Problem during pa...
Name: concat, dtype: object

In [16]:
data.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID', 'concat'],
      dtype='object')

In [17]:
categorias = ['Credit card','Mortgage', 'Student loan','Consumer Loan']

In [18]:
data=data.loc[(data['Product'] == categorias[0])| (data['Product'] == categorias[1])| (data['Product'] == categorias[2])| (data['Product'] == categorias[3])]
data


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,concat
31,2019-03-05,Mortgage,Conventional home mortgage,Trouble during payment process,,This complaint dates back to last XX/XX/XXXX d...,,NATIONSTAR MORTGAGE,FL,339XX,,Consent provided,Web,2019-03-11,Closed with explanation,Yes,,3170293,Trouble during payment process This complaint...
34,2019-07-30,Mortgage,Conventional home mortgage,Trouble during payment process,,,Company has responded to the consumer and the ...,"LoanCare, LLC",MI,48178,,Consent not provided,Web,2019-07-30,Closed with explanation,Yes,,3323588,Trouble during payment process
46,2019-11-15,Mortgage,Conventional home mortgage,Closing on a mortgage,,We have already tried to contact the company w...,,"ClosingMark Financial Services, LLC",CO,802XX,,Consent provided,Web,2019-11-15,Closed with explanation,Yes,,3439091,Closing on a mortgage We have already tried t...
55,2021-02-04,Mortgage,Conventional home mortgage,Trouble during payment process,,,,RP FUNDING INCORPORATED,FL,32092,,,Web,2021-02-04,In progress,Yes,,4116068,Trouble during payment process
57,2018-04-14,Mortgage,Conventional home mortgage,Trouble during payment process,,The mortgage company ( Roundpoint Mortgageg ) ...,Company has responded to the consumer and the ...,RoundPoint Mortgage Servicing Corporation,IL,600XX,,Consent provided,Web,2018-04-14,Closed with explanation,Yes,,2875337,Trouble during payment process The mortgage c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1959674,2017-01-19,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Received bad information about my loan,Over the life of my Student Loans through XXXX...,,"Navient Solutions, LLC.",TX,770XX,,Consent provided,Web,2017-01-19,Closed with explanation,Yes,No,2298947,Dealing with my lender or servicer Received ba...
1959675,2016-07-11,Consumer Loan,Vehicle loan,Managing the loan or lease,,I was on automatic payment for my car loan. In...,,NISSAN MOTOR ACCEPTANCE CORPORATION,IL,607XX,,Consent provided,Web,2016-07-11,Closed with explanation,Yes,No,2007463,Managing the loan or lease I was on automatic...
1959677,2015-03-26,Mortgage,FHA mortgage,"Loan servicing, payments, escrow account",,"On XXXX XXXX, 2015, I contacted XXXX XXXX, who...",,Finance of America Mortgage LLC,CA,925XX,,Consent provided,Web,2015-03-26,Closed with monetary relief,Yes,No,1303324,"Loan servicing, payments, escrow account On X..."
1959678,2015-12-12,Mortgage,Conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",,I can not get from chase who services my mortg...,,JPMORGAN CHASE & CO.,NY,110XX,,Consent provided,Web,2015-12-12,Closed with explanation,Yes,No,1694674,"Loan servicing, payments, escrow account I ca..."


In [19]:
data_train = data[['concat','Product']]

In [20]:
X = data_train['concat']
y = data_train['Product']
y

31              Mortgage
34              Mortgage
46              Mortgage
55              Mortgage
57              Mortgage
               ...      
1959674     Student loan
1959675    Consumer Loan
1959677         Mortgage
1959678         Mortgage
1959679      Credit card
Name: Product, Length: 501025, dtype: object

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [22]:
X_train[:10]

1036714           Loan servicing, payments, escrow account  
1275281                       Settlement process and costs  
564851     Applying for a mortgage or refinancing an exis...
1452835                Problems when you are unable to pay  
792511                      Trouble during payment process  
294018                          Struggling to pay mortgage  
1204417           Loan servicing, payments, escrow account  
1586930    Billing disputes  I applied and received a car...
1650391    Struggling to pay mortgage  Due to the Covid19...
942014                      Trouble during payment process  
Name: concat, dtype: object

In [23]:
y_train[:10]

1036714         Mortgage
1275281         Mortgage
564851          Mortgage
1452835    Consumer Loan
792511          Mortgage
294018          Mortgage
1204417         Mortgage
1586930      Credit card
1650391         Mortgage
942014          Mortgage
Name: Product, dtype: object

#Tokenização de texto

##Com scikit-learn Pré-processamento de texto, tokenização e filtragem de palavras irrelevantes estão todos incluídos no CountVectorizer, que cria um dicionário de recursos e transforma documentos em vetores de recursos:

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts
X_train_counts.shape


(375768, 66791)

##CountVectorizer suporta contagens de N-gramas de palavras ou caracteres consecutivos. Uma vez ajustado, o vetorizador construiu um dicionário de índices de recursos:

In [25]:
count_vect.vocabulary_.get(u'card')

11775

##O valor do índice de uma palavra no vocabulário está relacionado à sua frequência em todo o corpus de treinamento

##De ocorrências a frequências

##A contagem de ocorrências é um bom começo, mas há um problema: documentos mais longos terão valores médios de contagem mais altos do que documentos mais curtos, embora possam falar sobre os mesmos tópicos. Para evitar essas discrepâncias em potencial, é suficiente dividir o número de ocorrências de cada palavra em um documento pelo número total de palavras no documento: esses novos recursos são chamados de tf para Frequências de termo. Outro refinamento além do tf é reduzir os pesos das palavras que ocorrem em muitos documentos no corpus e, portanto, são menos informativas do que aquelas que ocorrem apenas em uma parte menor do corpus. Essa redução é chamada de tf – idf para “Frequência do termo vezes frequência inversa do documento”. Tanto tf quanto tf – idf podem ser calculados da seguinte maneira usando TfidfTransformer:

In [26]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(375768, 66791)

#Treinando um classificador 

##Agora que temos nossos recursos, podemos treinar um classificador para tentar prever a categoria de uma postagem. Vamos começar com um classificador Nayve-Bayes, que fornece uma boa base para esta tarefa. O scikit-learn inclui várias variantes desse classificador; o mais adequado para contagem de palavras é a variante multinomial:

In [27]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

##Para tentar prever o resultado em um novo documento, precisamos extrair os recursos usando quase a mesma cadeia de extração de recursos de antes. A diferença é que chamamos transform em vez de fit_transform nos transformadores, uma vez que eles já foram ajustados ao conjunto de treinamento:

In [28]:
complaint = ['Received bad information about my loan', 'I did not received my Credict Card']
X_new_counts = count_vect.transform(complaint)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for comp, category in zip(complaint, predicted):
    print('%r => %s' % (comp, category))

'Received bad information about my loan' => Student loan
'I did not received my Credict Card' => Credit card


#Construindo um pipeline

## Para tornar o vectorizer => transformer => classifier mais fácil de trabalhar, o scikit-learn fornece uma classe Pipeline que se comporta como um classificador composto:

In [29]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

##Os nomes vect, tfidf e clf (classificador) são arbitrários. Iremos usá-los para realizar o grid search para hiperparâmetros adequados abaixo. Agora podemos treinar o modelo com um único comando:

In [30]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

#Avaliação do desempenho no conjunto de teste



In [31]:
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.9572239475638088

##Buscando melhorar performance com máquina de vetores de suporte linear (SVM), que é amplamente considerada como um dos melhores algoritmos de classificação de texto (embora também seja um pouco mais lenta do que o Nayve-Bayes). Podemos mudar o aluno simplesmente conectando um objeto classificador diferente em nosso pipeline:


In [32]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

text_clf.fit(X_train,y_train)

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.9485777241990467

##O scikit-learn fornece mais utilitários para uma análise de desempenho mais detalhada dos resultados:

In [33]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))

               precision    recall  f1-score   support

Consumer Loan       0.93      0.76      0.84      7887
  Credit card       0.92      0.95      0.93     22247
     Mortgage       0.95      0.98      0.97     79920
 Student loan       1.00      0.88      0.93     15203

     accuracy                           0.95    125257
    macro avg       0.95      0.89      0.92    125257
 weighted avg       0.95      0.95      0.95    125257



In [34]:
metrics.confusion_matrix(y_test, predicted)

array([[ 5986,   184,  1666,    51],
       [    0, 21036,  1210,     1],
       [    0,  1493, 78421,     6],
       [  458,    67,  1305, 13373]])

##Aplicando Grid Search

In [35]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3),
}

In [36]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

##A instância do Grid Search se comporta como um modelo normal de scikit-learn. Vamos realizar a pesquisa em um subconjunto menor dos dados de treinamento para acelerar o cálculo:

In [37]:
gs_clf = gs_clf.fit(X_train[:10000], y_train[:10000])


In [38]:
gs_clf.best_score_

0.9488999999999999

In [39]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 1)
